In [1]:
import os
import pandas as pd


df = pd.read_csv(os.environ['MIMIC_EHR_PIPELINE_BASE'] + '''/data/sepsis_micro_cohort/final/data_matrix.csv''')

In [5]:
df

,person_id,visit_occurrence_id,measurement_date,visit_start_datetime_adm,death_datetime_adm,death_0_7_adm,death_0_14_adm,death_0_21_adm,death_0_30_adm,death_0_60_adm,...,labs_Prothrombin time (PT)_last,labs_INR in Platelet poor plasma by Coagulation assay_last,labs_aPTT in Blood by Coagulation assay_last,labs_Specific gravity of Urine by Test strip_last,labs_pH of Urine by Test strip_last,labs_Bilirubin.total in Serum or Plasma_last,labs_Aspartate aminotransferase in Serum or Plasma_last,labs_Alanine aminotransferase in Serum or Plasma_last,labs_Alkaline phosphatase in Serum or Plasma_last,labs_Albumin in Serum or Plasma_last
0,-1788739119,-2142878850,2159-05-24,2159-05-22 20:37:00,2159-09-18 00:00:00,0,0,0,0,0,...,15.9,1.5,32.500000,1.020300,6.620000,1.900000,38.000000,19.000000,154.000000,3.320500
1,-1788739119,-2142878850,2159-05-26,2159-05-22 20:37:00,2159-09-18 00:00:00,0,0,0,0,0,...,14.8,1.4,33.800000,1.016395,6.905000,2.328000,58.430000,43.331667,181.220000,2.836500
2,-1788739119,-2142878850,2159-05-27,2159-05-22 20:37:00,2159-09-18 00:00:00,0,0,0,0,0,...,12.5,1.1,35.900000,1.016577,6.820000,1.000000,27.000000,16.000000,151.000000,2.679500
3,-1788739119,-2142878850,2159-05-28,2159-05-22 20:37:00,2159-09-18 00:00:00,0,0,0,0,0,...,11.8,1.1,37.000000,1.015025,6.845000,2.486833,46.225000,67.945000,223.955000,2.695000
4,-1788739119,-2142878850,2159-05-29,2159-05-22 20:37:00,2159-09-18 00:00:00,0,0,0,0,0,...,12.2,1.1,40.841917,1.019545,6.675000,1.500000,54.995000,80.800000,240.260000,2.519500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5609,-195010687,2141255146,2126-08-24,2126-08-10 12:54:00,2134-04-12 00:25:00,0,0,0,0,0,...,13.7,1.2,24.900000,1.019160,6.675000,1.448000,76.670000,132.130000,212.636667,3.743500
5610,-195010687,2141255146,2126-08-25,2126-08-10 12:54:00,2134-04-12 00:25:00,0,0,0,0,0,...,14.1,1.2,25.600000,1.016830,6.614167,1.497500,79.383333,138.458333,199.287500,3.504500
5611,-939423329,2143021773,2169-04-05,2169-02-13 00:00:00,2169-07-04 01:17:00,0,0,0,0,0,...,16.3,1.5,29.200000,1.012567,5.750000,0.851500,27.248333,30.435000,161.455000,2.826667
5612,-939423329,2143021773,2169-04-06,2169-02-13 00:00:00,2169-07-04 01:17:00,0,0,0,0,0,...,14.6,1.4,32.000000,1.012815,5.782000,0.300000,10.000000,7.000000,50.000000,3.096500


In [6]:
df.death_0_7_adm.value_counts()

death_0_7_adm
0    5474
1     140
Name: count, dtype: int64

In [13]:
len(df.visit_occurrence_id.unique())

729

In [2]:
def filterWindow(dataDf, anchorDateColumn, measurementDateColumn, windowStart, windowEnd):

    import pandas as pd

    dataDf[anchorDateColumn] = pd.to_datetime(dataDf[anchorDateColumn])
    dataDf[measurementDateColumn] = pd.to_datetime(dataDf[measurementDateColumn])
    dataDf = dataDf[(dataDf[measurementDateColumn] >= (dataDf[anchorDateColumn] - pd.Timedelta(days=windowStart))) & (dataDf[measurementDateColumn] <= (dataDf[anchorDateColumn] + pd.Timedelta(days=windowEnd)))]

    return dataDf


In [3]:
def readData(dirPath, idColumns=['id'], targetColumn='target', anchorDateColumn='visit_start_date_adm', measurementDateColumn='measurement_date', windowStart=0, windowEnd=3):

    import pandas as pd

    dataDf = pd.read_csv(dirPath)

    dataDf = filterWindow(
        dataDf=dataDf,
        anchorDateColumn=anchorDateColumn,
        measurementDateColumn=measurementDateColumn,
        windowStart=windowStart,
        windowEnd=windowEnd
        )

    print(dataDf.shape)

    XVitalsAvg = dataDf[idColumns + [measurementDateColumn] + [col for col in dataDf.columns if (col.startswith('vitals_') and col.endswith('_avg'))]]
    XVitalsMin = dataDf[idColumns + [measurementDateColumn] + [col for col in dataDf.columns if (col.startswith('vitals_') and col.endswith('_min'))]]
    XVitalsMax = dataDf[idColumns + [measurementDateColumn] + [col for col in dataDf.columns if (col.startswith('vitals_') and col.endswith('_max'))]]
    XVitalsFirst = dataDf[idColumns + [measurementDateColumn] + [col for col in dataDf.columns if (col.startswith('vitals_') and col.endswith('_first'))]]
    XVitalsLast = dataDf[idColumns + [measurementDateColumn] + [col for col in dataDf.columns if (col.startswith('vitals_') and col.endswith('_last'))]]
    XLabsAvg = dataDf[idColumns + [measurementDateColumn] + [col for col in dataDf.columns if (col.startswith('labs_') and col.endswith('_avg'))]]
    XLabsMin = dataDf[idColumns + [measurementDateColumn] + [col for col in dataDf.columns if (col.startswith('labs_') and col.endswith('_min'))]]
    XLabsMax = dataDf[idColumns + [measurementDateColumn] + [col for col in dataDf.columns if (col.startswith('labs_') and col.endswith('_max'))]]
    XLabsFirst = dataDf[idColumns + [measurementDateColumn] + [col for col in dataDf.columns if (col.startswith('labs_') and col.endswith('_first'))]]
    XLabsLast = dataDf[idColumns + [measurementDateColumn] + [col for col in dataDf.columns if (col.startswith('labs_') and col.endswith('_last'))]]

    y = dataDf[idColumns + [targetColumn]]
    
    print(XVitalsAvg.shape, XLabsAvg.shape)

    XVitalsAvgAgg = XVitalsAvg.groupby(idColumns).mean().reset_index().drop(columns=measurementDateColumn)
    XVitalsMinAgg = XVitalsMin.groupby(idColumns).min().reset_index().drop(columns=measurementDateColumn)
    XVitalsMaxAgg = XVitalsMax.groupby(idColumns).max().reset_index().drop(columns=measurementDateColumn)
    XVitalsFirstAgg = XVitalsFirst.sort_values(idColumns + [measurementDateColumn], ascending=True).groupby(idColumns).first().reset_index().drop(columns=measurementDateColumn)
    XVitalsLastAgg = XVitalsLast.sort_values(idColumns + [measurementDateColumn], ascending=False).groupby(idColumns).first().reset_index().drop(columns=measurementDateColumn)
    XLabsAvgAgg = XLabsAvg.groupby(idColumns).mean().reset_index().drop(columns=measurementDateColumn)
    XLabsMinAgg = XLabsMin.groupby(idColumns).min().reset_index().drop(columns=measurementDateColumn)
    XLabsMaxAgg = XLabsMax.groupby(idColumns).max().reset_index().drop(columns=measurementDateColumn)
    XLabsFirstAgg = XLabsFirst.sort_values(idColumns + [measurementDateColumn], ascending=True).groupby(idColumns).first().reset_index().drop(columns=measurementDateColumn)
    XLabsLastAgg = XLabsLast.sort_values(idColumns + [measurementDateColumn], ascending=False).groupby(idColumns).first().reset_index().drop(columns=measurementDateColumn)
    yAgg = y.groupby(idColumns).first().reset_index()

    X = XVitalsAvgAgg\
        .merge(XVitalsMinAgg, on=idColumns, how='outer')\
        .merge(XVitalsMaxAgg, on=idColumns, how='outer')\
        .merge(XVitalsFirstAgg, on=idColumns, how='outer')\
        .merge(XVitalsLastAgg, on=idColumns, how='outer')\
        .merge(XLabsAvgAgg, on=idColumns, how='outer')\
        .merge(XLabsMinAgg, on=idColumns, how='outer')\
        .merge(XLabsMaxAgg, on=idColumns, how='outer')\
        .merge(XLabsFirstAgg, on=idColumns, how='outer')\
        .merge(XLabsLastAgg, on=idColumns, how='outer')

    XVitalsAvgAgg = XVitalsAvgAgg.drop(columns=idColumns)
    XVitalsMinAgg = XVitalsMinAgg.drop(columns=idColumns)
    XVitalsMaxAgg = XVitalsMaxAgg.drop(columns=idColumns)
    XVitalsFirstAgg = XVitalsFirstAgg.drop(columns=idColumns)
    XVitalsLastAgg = XVitalsLastAgg.drop(columns=idColumns)
    XLabsAvgAgg = XLabsAvgAgg.drop(columns=idColumns)
    XLabsMinAgg = XLabsMinAgg.drop(columns=idColumns)
    XLabsMaxAgg = XLabsMaxAgg.drop(columns=idColumns)
    XLabsFirstAgg = XLabsFirstAgg.drop(columns=idColumns)
    XLabsLastAgg = XLabsLastAgg.drop(columns=idColumns)
    XAgg = X.drop(columns=idColumns)
    idsDf =yAgg[idColumns]
    yAgg = yAgg.drop(columns=idColumns)

    return XAgg, XVitalsAvgAgg, XVitalsMinAgg, XVitalsMaxAgg, XVitalsFirstAgg, XVitalsLastAgg, XLabsAvgAgg, XLabsMinAgg, XLabsMaxAgg, XLabsFirstAgg, XLabsLastAgg, yAgg, idsDf


In [4]:
data = readData(
    dirPath=os.environ['MIMIC_EHR_PIPELINE_BASE'] + '''/data/sepsis_micro_cohort/final/data_matrix.csv''',
    idColumns=['person_id', 'visit_occurrence_id'],
    targetColumn='death_0_7_adm',
    measurementDateColumn='measurement_date',
    anchorDateColumn='visit_start_datetime_adm',
    windowStart=0,
    windowEnd=3,
    )

X, XVitalsAvg, XVitalsMin, XVitalsMax, XVitalsFirst, XVitalsLast, XLabsAvg, XLabsMin, XLabsMax, XLabsFirst, XLabsLast, y, idsDf = data


(654, 278)
(654, 20) (654, 38)


In [5]:
df.shape, len(df.visit_occurrence_id.unique()), X.shape, XVitalsAvg.shape, XLabsAvg.shape

((5614, 278), 729, (369, 260), (369, 17), (369, 35))

In [6]:
X

,vitals_Respiratory rate_avg,vitals_Heart rate_avg,vitals_Body weight_avg,vitals_Oxygen saturation in Arterial blood by Pulse oximetry_avg,vitals_Glasgow coma score eye opening_avg,vitals_Glasgow coma score verbal_avg,vitals_Glasgow coma score motor_avg,vitals_Physical mobility Braden scale_avg,vitals_Nutrition intake pattern Braden scale_avg,vitals_Sensory perception Braden scale_avg,...,labs_Prothrombin time (PT)_last,labs_INR in Platelet poor plasma by Coagulation assay_last,labs_aPTT in Blood by Coagulation assay_last,labs_Specific gravity of Urine by Test strip_last,labs_pH of Urine by Test strip_last,labs_Bilirubin.total in Serum or Plasma_last,labs_Aspartate aminotransferase in Serum or Plasma_last,labs_Alanine aminotransferase in Serum or Plasma_last,labs_Alkaline phosphatase in Serum or Plasma_last,labs_Albumin in Serum or Plasma_last
0,16.960000,114.320000,192.746000,96.960000,4.000000,5.000000,6.000000,3.000000,3.000000,3.00,...,72.935417,7.418583,66.800000,1.012670,6.222500,2.076500,52.630000,51.970000,272.676667,2.587000
1,18.727273,68.120000,156.484667,99.960000,2.833333,1.000000,5.666667,2.833333,1.000000,3.00,...,14.100000,1.200000,32.700000,1.018355,5.405000,3.280083,58.801667,60.356667,306.850000,2.002167
2,22.872340,89.875000,159.500000,96.346154,1.571429,1.000000,2.000000,1.000000,1.600000,1.00,...,17.080000,1.560000,64.800000,1.013925,5.292500,0.600000,136.000000,116.000000,59.000000,3.366667
3,16.921131,96.979167,210.301750,98.973732,3.095238,1.000000,4.869048,1.900000,2.000000,2.40,...,70.588167,6.870500,64.640333,1.019775,6.699167,1.935000,58.340000,46.070000,269.655000,2.377000
4,23.071304,114.303478,129.903500,95.257391,4.000000,5.000000,6.000000,3.000000,1.625000,4.00,...,20.800000,1.900000,39.300000,1.025000,6.000000,4.310833,110.100000,85.847500,183.388667,2.580500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,17.187500,85.500000,154.000000,95.312500,3.000000,4.000000,6.000000,2.000000,2.000000,2.00,...,12.300000,1.100000,30.014667,1.006000,5.500000,0.590083,71.645000,101.240000,132.815000,3.366833
365,25.786834,106.818519,151.675333,97.022103,3.875000,5.000000,6.000000,2.500000,2.000000,2.75,...,10.400000,0.950000,29.050000,1.017405,6.257500,1.600000,106.000000,111.000000,72.000000,2.635500
366,21.091270,105.379831,209.000000,98.379227,3.166667,3.833333,5.722222,2.033333,2.466667,2.80,...,13.300000,1.200000,40.233333,1.018330,5.620000,1.672667,41.125000,41.335000,142.807500,2.345000
367,20.916667,106.000000,140.450000,97.916667,2.500000,2.000000,5.000000,2.000000,2.000000,3.00,...,12.600000,1.200000,29.500000,1.011825,6.027500,0.829083,59.015000,74.338333,141.653333,3.172000


In [7]:
y

,death_0_7_adm
0,0
1,0
2,1
3,0
4,0
...,...
364,1
365,0
366,0
367,0


In [8]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate


xgb = XGBClassifier(use_label_encoder=False)

xgbScores = cross_validate(xgb, X, y, cv=5, scoring=['accuracy', 'balanced_accuracy',  'average_precision', 'f1', 'roc_auc'])
xgbScores

{'fit_time': array([0.36748886, 0.33486557, 0.32897496, 0.32256866, 0.35034871]),
 'score_time': array([0.09237981, 0.07142282, 0.07116127, 0.07607365, 0.06982398]),
 'test_accuracy': array([0.85135135, 0.86486486, 0.86486486, 0.86486486, 0.8630137 ]),
 'test_balanced_accuracy': array([0.4921875, 0.5      , 0.5      , 0.5      , 0.5      ]),
 'test_average_precision': array([0.13950897, 0.1310163 , 0.12970118, 0.19258623, 0.13413189]),
 'test_f1': array([0., 0., 0., 0., 0.]),
 'test_roc_auc': array([0.4953125 , 0.4421875 , 0.41875   , 0.5953125 , 0.46666667])}